**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Data Preprocessing

In the previous examples we have made use of the Iris dataset. Given that this dataset only contains 4 columns and all the data is numeric with very similar scales it was not necessary to apply any special kind of preprocessing.

In practice, this is very rarely the case. The prepration of the dataset usually takes significantly more developer time than the application and the tuning of the model itself. It is typically necessary to clean the data, to handle missing values, rescale, encode categorical variables in a different form, etc.

This notebook will show how some of the most fundamental types of preprocessing work and how to design the preprocessing phase in a way that will enable us to easily reproduce it for new data.

The overall procedure when training a model could be summarized as follows:

* Load the dataset.
* Split into train and test set (if not split already).
* Clean and preprocess the data, e.g.:* Handle missing values.
* Rescale numeric data appropriately.
* Encode categorical variable numerically.

* Train a model on the training set.
* Test generalization on the test set.
### Loading the Titanic Dataset

As a preprocessing example we will now use the well-known [Titanic](https://www.kaggle.com/c/titanic) dataset. The dataset contains data about the passangers of Titanic. The goal is to predict which passengers have survived the tragedy and which have not. In order to get some idea of what kind of data we will be working with, let us first display a short description contained in the file `description.txt`:



In [ ]:
with open("data/titanic/description", "r") as file:
    print("".join(file.readlines()))

Next we can read the dataset itself from a CSV file and split it into the train and test set. We will stratify by the class (i.e. by whether the passenger has survived or not):



In [ ]:
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

### Simple Preprocessing

#### Rescaling Numeric Inputs

With many machine learning methods it is a good idea to rescale numeric data into some standard scale before use – otherwise the method might put more emphasis on data with a larger relative scale, which is typically not desirable. There are multiple types of such rescaling, for instance:

* Rescaling into the interval from 0 to 1 (e.g. using `sklearn.preprocessing.MinMaxScaler`);
* Standardizing the data (shift the mean to zero and rescale the variance to 1; `sklearn.preprocessing.StandardScaler`);
* ...
We will show an example of standardization (other types of scaling can be applied in analogical ways) – let us suppose that we would like to standardize column `Fare`. In the `scikit-learn` package, which we will be using, all similar operations are done using a common unified interace – so called transformers. Every transformer is first constructed and then its parameters can be tuned using the `fit` method:



In [ ]:
scaler = StandardScaler()
scaler.fit(df_train[['Fare']])

Transformed data can be obtained using the `transform` method:



In [ ]:
fare_scaled = scaler.transform(df_train[['Fare']])

If we intend to use the data to fit the transformer, but we also want the tranformer to rescale the same data, there is also a combined method called `fit_transform` – in our case it will be better to use this one:



In [ ]:
scaler = StandardScaler()
fare_scaled = scaler.fit_transform(df_train[['Fare']])

Let us now display a portion of the original and the transformed column to make sure that our transformer works:



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Fare']], fare_scaled]),
    columns=["Fare", "Fare Scaled"]
).head()

We can also inspect the means and variances to determine whether they changed in the way we expected:



In [ ]:
print("Mean of fare: {}\nVariance of fare: {}".format(
    np.mean(df_train['Fare']),
    np.var(df_train['Fare'])
))

In [ ]:
print("Mean of scaled fare: {}\nVariance of scaled fare: {}".format(
    np.mean(fare_scaled),
    np.var(fare_scaled)
))

#### Re-encoding the Categorical Variables

Datasets will often contain categorical variables, domains of which are discrete and relatively small and the values are typically represented by strings. In our case, `Embarked` (the port at which the passenger had embarked) is one such variable and `Sex` (the sex of the passenger) is another.

Depending on the implementation of our machine learning method, it may be necessary to transform such variables from textual to numeric representation (by assigning some numeric value to every distinct string value). In Python this can be done in a way very similar to rescaling numeric attributes – we only need to use a different transformer: `OrdinalEncoder`.



In [ ]:
ordenc = OrdinalEncoder()
sex_encoded = ordenc.fit_transform(df_train[["Sex"]])

For comparison we can again display the original and the the transformed column:



In [ ]:
pd.DataFrame(
    np.hstack([df_train[["Sex"]], sex_encoded]),
    columns=["Sex", "Sex Encoded"]
).head()

As we can see, value `female` was re-encoded as 0 and value `male` as 1. We can also ascertain that this is the case by comparing their relative positions in the following list:



In [ ]:
ordenc.categories_

### Handling Missing Values

If we try to apply the procedures that we have just outlined to the remaining columns in our dataset, we will find out that this does not always work. Datasets will very commonly have missing values (i.e. some columns will not be filled in for a particular entry). For the majority of machine learning methods this will consitute a problem and we will we need to handle missing values in some way prior to applying the method. In principle, there are three groups of approaches to this problem:

* To drop rows with missing values altogether.
* Fill in the missing values according to some rule (imputation);
* To leave the missing values as they are, if the machine learning method can handle them direclty (e.g. some implementations of decision trees can).
Entire rows are usually only dropped when we can reasonably expect them to contain very little useful information (e.g. when almost all of the values are missing) or when we have so much data, that we can afford not to use incomplete entries (which happens very rarely).

The procedures used to imputate missing values can vary in complexity:

* Very simple – e.g. the missing values are replaced by the average or the most frequent value from the corresponding column.
* Very complex – e.g. train a full model using some of the other columns and use that model to impute the missing values.
* Moderately complex...
We will only show one trivial way of imputing the values for numeric and for categorical data (we will use class `SimpleImputer` in both cases), but more ways can be found in the `scikit-learn` package and elsewhere (e.g. `sklearn.impute.IterativeImputer`).

#### Detection of Missing Values

But before imputing the missing values, how do we even detect that a column has missing values in the first place. The `pandas` package has an `.isnull()`, method for just this purpose – it returns whether there is a missing value or not for each single cell:



In [ ]:
df_train["Age"].isnull()[:10]

If we want to know whether a particular column has at least one missing value, we can chain `isnull()` with `.any()`:



In [ ]:
df_train["Age"].isnull().any()

In [ ]:
df_train["Fare"].isnull().any()

Ako vidno, v stĺpci `Age` niektoré hodnoty chýbajú, ale napríklad v stĺpci `Fare` máme údaje pre každého pasažiera.

#### Triviálne doplnenie numerických hodnôt

Triviálne doplnenie chýbajúcich hodnôt v numerickom stĺpci sa dá realizovať napríklad pomocou transformátora `SimpleImputer`. Ten predvolene dopĺňa chýbajúce hodnoty strednou hodnotou daného stĺpca, dajú sa však zvoliť aj iné stratégie:



In [ ]:
num_impute = SimpleImputer()
age_imputed = num_impute.fit_transform(df_train[["Age"]])

The result will be as follows:



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Age']], age_imputed]),
    columns=["Age", "Age Imputed"]
).head()

#### Imputation of Categorical Attributes

To impute the missing values for categorical attributes we can again use the `SimpleImputer` transformer. However, we will need to parametrize is differently. If we want to replace missing values by the most frequent value for that column, we can use the following setup, for instance:



In [ ]:
cat_impute = SimpleImputer(strategy="most_frequent")
embarked_imputed = cat_impute.fit_transform(df_train[["Embarked"]])

Another alternative would be to add a new value – called `MISSING` or something similar – to the categorical attribute and use that to indicate that the value was missing:



In [ ]:
cat_impute = SimpleImputer(strategy='constant', fill_value='MISSING')
embarked_imputed = cat_impute.fit_transform(df_train[["Embarked"]])

### Reproducibility of Preprocessing

Naturally, having designed a preprocessing stage, we would like to be able to apply that same stage to testing set and later, once our model has been deployed, to all new data. One thing that we therefore need to consider is that we tune the parameters of our transformers on our data. We need to make sure that we use identically tuned transformers to preprocess our testing and other data, otherwise the results will be different and our model will not work correctly. For instance, the same categorical value could be encoded as 3 for the train set, but as 1 for the test set.

#### An Incorrect Way to Preprocess

We will now show a small example of how not to preprocess. Let's say that a programmer wanted to make their code reproducible and so they decided to wrap it in a function, which they would first call with their training data and then again with their testing data. However, they forgot that inside that function the parameters of the transformers get tuned and now they will be tuned differently for the training and the testing set.



In [ ]:
def preprocess(df):
    num_impute = SimpleImputer()
    age_imputed = num_impute.fit_transform(df[['Age']])
    
    scaler = StandardScaler()
    age_scaled = scaler.fit_transform(age_imputed)
    
    return age_scaled

In [ ]:
df_train_preproc = preprocess(df_train)
df_test_preproc = preprocess(df_test)

The output will be as follows. You should be able to see that value 26 was tranformed to a different number each time, which is, of course, unacceptable.



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Age']], df_train_preproc]),
    columns=["Age", "Scaled Age"]
).head()

In [ ]:
pd.DataFrame(
    np.hstack([df_test[['Age']], df_test_preproc]),
    columns=["Age", "Scaled Age"]
).head()

#### The Correct Procedure

In order to get correct results, one needs to apply the same trasformer – tuned on the training set – to the testing set. This could be done in the following way for instance:



In [ ]:
def preprocess(df, params=None):
    if params is None:
        params = {}
        
        params["num_impute"] = SimpleImputer()
        age_imputed = params["num_impute"].fit_transform(df[['Age']])
        
        params["scaler"] = StandardScaler()
        age_scaled = params["scaler"].fit_transform(age_imputed)
        
    else:
        age_imputed = params["num_impute"].transform(df[['Age']])
        age_scaled = params["scaler"].transform(age_imputed)
        
    return age_scaled, params

In [ ]:
df_train_preproc, params = preprocess(df_train)
df_test_preproc, params = preprocess(df_test, params)

Given that we now store both transformers and we make sure to apply them to the testing data using the `transform` method, the results should be correct now.



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Age']], df_train_preproc]),
    columns=["Age", "Scaled Age"]
).head()

In [ ]:
pd.DataFrame(
    np.hstack([df_test[['Age']], df_test_preproc]),
    columns=["Age", "Scaled Age"]
).head()

#### A Simpler Approach: Scikit-learn Pipelines

The problem of the approach that we have just outlined is that in practice preprocessing can be quite complex. To manually keep track of all the transformers that have been used and to make sure that they are applied in exactly the same way afterwards is a relatively daunting task – and one easily prone to error. For this reason we will use the next notebook to show how this process can be automated using the so-called **pipelines**  from the `scikit-learn` package.

